In [ ]:
import pybullet as p
import pybullet_data
import time
import math

# Initialize PyBullet simulation
def initialize_simulation():
    p.connect(p.GUI)  # Use p.DIRECT for headless mode
    p.setAdditionalSearchPath(pybullet_data.getDataPath())  # Path to PyBullet's data
    p.setGravity(0, 0, -20) 
    
    plane_id = p.loadURDF("plane.urdf")  # Load a flat ground plane
    return plane_id

# Load the robot URDF
def load_robot(urdf_path):
    start_position = [0, 0, 0.3]  # Initial position
    start_orientation = p.getQuaternionFromEuler([0, 0, 0])
    p.setAdditionalSearchPath("quadruped_robot/urdf/")
    # Add friction to the fee
   

    robot_id = p.loadURDF("quadruped_robot.urdf", start_position, start_orientation, useFixedBase=False)
    num_joints = p.getNumJoints(robot_id)
    for joint_index in range(num_joints):
        # Adjust lateral friction and damping for each joint
        p.changeDynamics(robot_id, joint_index, lateralFriction=1.0, spinningFriction=0.1, rollingFriction=0.1, linearDamping=0.04, angularDamping=0.04)


    return robot_id

# Define a simple trot gait
# Alternate diagonally opposite legs: front-right & back-left, then front-left & back-right
def trot_gait(robot, time_step, amplitude=0.3, frequency=1.0):
    num_joints = p.getNumJoints(robot)

    # Define leg joint indices (modify based on your robot's joint configuration)
    front_right = [1, 2]  # Hip and knee joint indices for front-right leg
    back_left = [10, 11]    # Hip and knee joint indices for back-left leg
    front_left = [7, 8]   # Hip and knee joint indices for front-left leg
    back_right = [4, 5]   # Hip and knee joint indices for back-right leg

    # Phase offset for alternating legs
    phase_offset = math.pi

    # Calculate joint positions using sine wave
    for joint in front_right + back_left:
        target_position = amplitude * math.sin(2 * math.pi * frequency * time_step)
        p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=target_position,force=50, positionGain=0.03, velocityGain=0.03)

    for joint in front_left + back_right:
        target_position = amplitude * math.sin(2 * math.pi * frequency * time_step + phase_offset)
        p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=target_position,force=50, positionGain=0.03, velocityGain=0.03)

# Main simulation loop
def run_simulation(robot):
    time_step = 1 / 240  # PyBullet default timestep
    current_time = 0

    while True:
        # Apply the trot gait
        trot_gait(robot, current_time)

        # Step the simulation
        p.stepSimulation()
        time.sleep(time_step)  # Maintain real-time simulation

        # Update time
        current_time += time_step

# Main function
def main():
    initialize_simulation()
    robot = load_robot("quadruped_robot.urdf")  # Replace with your URDF file path
 
    run_simulation(robot)

if __name__ == "__main__":
    main()


pybullet build time: Nov 28 2023 23:45:17


In [ ]:
import pybullet as p
import pybullet_data
import time
import math

# Initialize PyBullet simulation
def initialize_simulation():
    p.connect(p.GUI)  # Use p.DIRECT for headless mode
    p.setAdditionalSearchPath(pybullet_data.getDataPath())  # Path to PyBullet's data
    p.setGravity(0, 0, -9.8)  # Adjust gravity
    
    plane_id = p.loadURDF("plane.urdf")  # Load a flat ground plane
    return plane_id

# Load the robot URDF
def load_robot(urdf_path):
    start_position = [0, 0, 0.3]  # Initial position
    start_orientation = p.getQuaternionFromEuler([0, 0, 0])
    p.setAdditionalSearchPath("quadruped_robot/urdf/")
    # Add friction to the feet
    robot_id = p.loadURDF("quadruped_robot.urdf", start_position, start_orientation, useFixedBase=False)
    num_joints = p.getNumJoints(robot_id)
    for joint_index in range(num_joints):
        # Adjust lateral friction and damping for each joint
        p.changeDynamics(robot_id, joint_index, lateralFriction=1.0, spinningFriction=0.1, rollingFriction=0.1, linearDamping=0.04, angularDamping=0.04)

    return robot_id

# Define a simple trot gait with added translation
# Alternate diagonally opposite legs: front-right & back-left, then front-left & back-right
def trot_gait(robot, time_step, amplitude=0.3, frequency=1.0):
    num_joints = p.getNumJoints(robot)

    # Define leg joint indices (modify based on your robot's joint configuration)
    front_right = [1, 2]  # Hip and knee joint indices for front-right leg
    back_left = [10, 11]    # Hip and knee joint indices for back-left leg
    front_left = [7, 8]   # Hip and knee joint indices for front-left leg
    back_right = [4, 5]   # Hip and knee joint indices for back-right leg

    # Phase offset for alternating legs
    phase_offset = math.pi

    # Calculate joint positions using sine wave
    for joint in front_right + back_left:
        target_position = amplitude * math.sin(2 * math.pi * frequency * time_step)
        p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=target_position, force=50, positionGain=0.03, velocityGain=0.03)

    for joint in front_left + back_right:
        target_position = amplitude * math.sin(2 * math.pi * frequency * time_step + phase_offset)
        p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=target_position, force=50, positionGain=0.03, velocityGain=0.03)

# Update robot's base position to simulate trajectory in x and z direction
def update_robot_position(robot, time_step, amplitude=0.3, frequency=0.5):
    # Define simple linear motion in the x direction and sinusoidal motion in the z direction
    x_position = 0.5 * time_step  # Move forward in x direction over time
    z_position = amplitude * math.sin(2 * math.pi * frequency * time_step)  # Sinusoidal motion for jumping effect

    # Set the new position for the robot base (x, y, z)
    p.resetBasePositionAndOrientation(robot, [x_position, 0, z_position + 0.3], [0, 0, 0, 1])

# Main simulation loop
def run_simulation(robot):
    time_step = 1 / 240  # PyBullet default timestep
    current_time = 0

    while True:
        # Apply the trot gait
        trot_gait(robot, current_time)

        # Update the robot's position based on the trajectory function
        update_robot_position(robot, current_time)

        # Step the simulation
        p.stepSimulation()
        time.sleep(time_step)  # Maintain real-time simulation

        # Update time
        current_time += time_step

# Main function
def main():
    initialize_simulation()
    robot = load_robot("quadruped_robot.urdf")  # Replace with your URDF file path
    run_simulation(robot)

if __name__ == "__main__":
    main()


pybullet build time: Nov 28 2023 23:45:17
